# read currents/voltages from good runs
The time information of good runs is extracted from the .npy file created by rundb_list_expertflags.py 

In [2]:
import epics_archiver
import numpy as np
import pandas as pd
import matplotlib.dates as mdate
import datetime


module_list = [
                '1012','1082','1022','2042',
                '1062','1072','2052','1042','1052',
                '1071','1061','2051','1041','1051',
                '1011','1031','1081','2041','1021'
                #,'1032'
]

iv_list = ["sw-sub", "sw-dvdd","sw-refin", "dhp-io", "dhp-core", "dcd-dvdd", "dcd-avdd", "dcd-refin", "dcd-amplow",
                "bulk", "clear-off", "clear-on", "gate-off", "gate-on1", "gate-on2", "gate-on3",
                "source", "ccg1", "ccg2", "ccg3", "drift", "hv", "guard"]

# iv_list = ['sw-dvdd']

expnr = 14
timeinfo = np.load('/home/ye/HY_pxd_scripts/exp%i_run_time_info.npy'%expnr, allow_pickle=True).item()

save_to_file = True


#### Belle2 archiver
b2_archiver = epics_archiver.epics_archiver("172.22.16.120:17668")

#### PXD archiver
pxd_archiver = epics_archiver.epics_archiver("172.22.18.80:80")

#### SVD archiver
svd_archiver = epics_archiver.epics_archiver("172.22.18.120:17668") 

iv_data={}
iv_data['run']=[]
iv_data['start_time']=[]
for i in module_list:
    for iv in iv_list:
        iv_data['%s_%s_c'%(i,iv)] = []
        iv_data['%s_%s_vload'%(i,iv)] = []

    
for idx, run in enumerate(sorted(timeinfo.keys())):
    try:
        start_time = datetime.datetime.strptime(timeinfo[run][0], "%Y-%m-%dT%H:%M:%S.%fZ")
    except Exception as e:
        try:
            start_time = datetime.datetime.strptime(timeinfo[run][0], "%Y-%m-%dT%H:%M:%SZ")
        except Exception as e:
            print(start_time, e)
            continue

    try:
        stop_time = datetime.datetime.strptime(timeinfo[run][1], "%Y-%m-%dT%H:%M:%S.%fZ")
    except Exception as e:
        try:
            stop_time = datetime.datetime.strptime(timeinfo[run][1], "%Y-%m-%dT%H:%M:%SZ")
        except Exception as e:
            print(stop_time, e)
            continue

    if idx%100 == 0:
        print('exp%i, run%i'%(expnr,run), start_time, stop_time)
    
    if (stop_time - start_time) < datetime.timedelta(minutes=60):
        continue 

    iv_data['run'].append(run)
    iv_data['start_time'].append(start_time)
    
    pxd_archiver_start = pxd_archiver.date_parser(start_time + datetime.timedelta(minutes=20))
    pxd_archiver_stop = pxd_archiver.date_parser(stop_time - datetime.timedelta(minutes=20))
    
    try:
        allpeak = np.allclose(pxd_archiver.getData('PXD:B:PSC-State:cur:S',pxd_archiver_start, pxd_archiver_stop)['vals'],3)
    except Exception as e:
        allpeak = False 
        
#     print(run, start_time, stop_time, allpeak)
    
    for module in module_list:    
        if not allpeak:
            for iv in iv_list:
                iv_data['%s_%s_c'%(module,iv)].append(np.nan)
                iv_data['%s_%s_vload'%(module,iv)].append(np.nan)
            continue

        for iv in iv_list:
            try:
                iv_data['%s_%s_c'%(module,iv)].append(
                    np.nanmean(pxd_archiver.getData('PXD:P%s:%s:CURR:cur'%(module, iv), pxd_archiver_start, pxd_archiver_stop)['vals']))
                iv_data['%s_%s_vload'%(module,iv)].append(
                    np.nanmean(pxd_archiver.getData('PXD:P%s:%s-load:VOLT:cur'%(module, iv), pxd_archiver_start, pxd_archiver_stop)['vals']))
            except Exception as e:
                iv_data['%s_%s_c'%(module,iv)].append(np.nan)
                iv_data['%s_%s_vload'%(module,iv)].append(np.nan)
#                 print(run, module, iv, e)

for key in sorted(iv_data):
    print(len(iv_data[key]), key)
    
if save_to_file: 
    np.save('exp%i_iv_goodruns'%expnr, iv_data)

print(expnr,'done')


exp14, run694 2020-10-30 02:40:11.054911 2020-10-30 03:19:25.756251
exp14, run908 2020-11-07 22:25:26.312090 2020-11-07 22:27:49.491464
exp14, run1578 2020-11-30 10:36:35.661560 2020-11-30 10:45:44.795603
exp14, run1971 2020-12-11 17:25:20.119366 2020-12-11 20:15:55.158961
164 1011_bulk_c
164 1011_bulk_vload
164 1011_ccg1_c
164 1011_ccg1_vload
164 1011_ccg2_c
164 1011_ccg2_vload
164 1011_ccg3_c
164 1011_ccg3_vload
164 1011_clear-off_c
164 1011_clear-off_vload
164 1011_clear-on_c
164 1011_clear-on_vload
164 1011_dcd-amplow_c
164 1011_dcd-amplow_vload
164 1011_dcd-avdd_c
164 1011_dcd-avdd_vload
164 1011_dcd-dvdd_c
164 1011_dcd-dvdd_vload
164 1011_dcd-refin_c
164 1011_dcd-refin_vload
164 1011_dhp-core_c
164 1011_dhp-core_vload
164 1011_dhp-io_c
164 1011_dhp-io_vload
164 1011_drift_c
164 1011_drift_vload
164 1011_gate-off_c
164 1011_gate-off_vload
164 1011_gate-on1_c
164 1011_gate-on1_vload
164 1011_gate-on2_c
164 1011_gate-on2_vload
164 1011_gate-on3_c
164 1011_gate-on3_vload
164 1011_gua

164 2051_gate-on3_vload
164 2051_guard_c
164 2051_guard_vload
164 2051_hv_c
164 2051_hv_vload
164 2051_source_c
164 2051_source_vload
164 2051_sw-dvdd_c
164 2051_sw-dvdd_vload
164 2051_sw-refin_c
164 2051_sw-refin_vload
164 2051_sw-sub_c
164 2051_sw-sub_vload
164 2052_bulk_c
164 2052_bulk_vload
164 2052_ccg1_c
164 2052_ccg1_vload
164 2052_ccg2_c
164 2052_ccg2_vload
164 2052_ccg3_c
164 2052_ccg3_vload
164 2052_clear-off_c
164 2052_clear-off_vload
164 2052_clear-on_c
164 2052_clear-on_vload
164 2052_dcd-amplow_c
164 2052_dcd-amplow_vload
164 2052_dcd-avdd_c
164 2052_dcd-avdd_vload
164 2052_dcd-dvdd_c
164 2052_dcd-dvdd_vload
164 2052_dcd-refin_c
164 2052_dcd-refin_vload
164 2052_dhp-core_c
164 2052_dhp-core_vload
164 2052_dhp-io_c
164 2052_dhp-io_vload
164 2052_drift_c
164 2052_drift_vload
164 2052_gate-off_c
164 2052_gate-off_vload
164 2052_gate-on1_c
164 2052_gate-on1_vload
164 2052_gate-on2_c
164 2052_gate-on2_vload
164 2052_gate-on3_c
164 2052_gate-on3_vload
164 2052_guard_c
164 2052_

In [6]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import matplotlib.dates as mdate
import datetime
from os import mkdir
from os.path import isdir, exists


module_list = [
                '1012','1082','1022','2042',
                '1062','1072','2052','1042','1052',
                '1071','1061','2051','1041','1051',
                '1011','1031','1081','2041','1021'
                #,'1032'
]

iv_list = ["sw-sub", 'sw-dvdd', "sw-refin", "dhp-io", "dhp-core", "dcd-dvdd", "dcd-avdd", "dcd-refin", "dcd-amplow",
                "bulk", "clear-off", "clear-on", "gate-off", "gate-on1", "gate-on2", "gate-on3",
                "source", "ccg1", "ccg2", "ccg3", "drift", "hv", "guard"]

# module_list = ['1011','1012','1021','1072','1081','1082']
# iv_list = ['hv']
# t0 = datetime.datetime.strptime('2020-04-23', "%Y-%m-%d")


savefigs = True
xaxis_date = False

iv_unit={}
for iv in iv_list:
    if iv =='hv':
        iv_unit['%s_c'%iv] = 'uA'
    else:
        iv_unit['%s_c'%iv] = 'mA'
    iv_unit['%s_v'%iv] = 'mV'


expnr = 14
iv_dict = np.load('exp%s_iv_goodruns.npy'%expnr, allow_pickle=True).item()
foldername = './exp%s_ps_good_runs'%expnr


iv_data = pd.DataFrame(iv_dict)

color=['tab:blue','tab:red', 'forestgreen']


for module in module_list:
    if savefigs and not exists('%s_%s'%(foldername, module)):
        mkdir('%s_%s'%(foldername, module))
    print( module)
    
    for iv in iv_list:

        subsample = iv_data.loc[(iv_data['%s_%s_c'%(module,iv)]!=np.nan) & (iv_data['%s_%s_vload'%(module,iv)]!=np.nan)]
#         subsample = iv_data.loc[(iv_data['%s_%s_c'%(module,iv)]!=np.nan) & (iv_data['%s_%s_vload'%(module,iv)]!=np.nan) & (iv_data['start_time']>t0)]

#         print( iv, subsample['start_time'][0], subsample['run'][0], subsample['start_time'][subsample.index[-1]], subsample['run'][subsample.index[-1]])
    
        fig0, ax0 = plt.subplots(figsize=(10,6), sharex=True, tight_layout=True)
        ax0.set_title('module_%s %s exp%i'%(module,iv,expnr))
        if xaxis_date:
            ax0.plot_date([mdate.date2num(t) for t in subsample['start_time']], subsample['%s_%s_c'%(module,iv)],c=color[0],ls='',marker='.', label='%s_%s_c'%(module,iv))
        else:
            ax0.plot(subsample['run'], subsample['%s_%s_c'%(module,iv)],c=color[0],ls='',marker='.', label='%s_%s_c'%(module,iv))

        ax0.set_ylabel('%s I_%s [%s]'%(module,iv,iv_unit['%s_c'%iv]), color=color[0])
        ax0.tick_params(axis='y', labelcolor=color[0])
        ax0.grid()
        imean = np.mean(subsample['%s_%s_c'%(module,iv)])
        istd = np.std(subsample['%s_%s_c'%(module,iv)])
        ax0.set_ylim(imean-8*istd, imean+5*istd)
#         ax0.set_ylim(-2000,0)
        if xaxis_date: 
            fig0.autofmt_xdate()
        else:
            ax0.set_xlabel('run')


        ax1 = ax0.twinx()
        if xaxis_date:
            ax1.plot_date([mdate.date2num(t) for t in subsample['start_time']], subsample['%s_%s_vload'%(module,iv)],c=color[1],ls='',marker='.', label='%s_%s_vload'%(module,iv))
        else:
            ax1.plot(subsample['run'], subsample['%s_%s_vload'%(module,iv)],c=color[1],ls='',marker='.', label='%s_%s_vload'%(module,iv))
        ax1.tick_params(axis='y', labelcolor=color[1])
        ax1.set_ylabel('%s V_%s at load [%s]'%(module,iv,iv_unit['%s_v'%iv]), color=color[1])
        vmean = np.mean(subsample['%s_%s_vload'%(module,iv)])
        vstd = np.std(subsample['%s_%s_vload'%(module,iv)])
        ax1.set_ylim(vmean-5*vstd, vmean+8*vstd)
#         ax1.set_ylim(-75000,-50000)

        if savefigs:
            if xaxis_date:
                plt.savefig('%s_%s/%s_%s_date.png'%(foldername,module,module,iv))
            else:
                plt.savefig('%s_%s/%s_%s_run.png'%(foldername,module,module,iv))

        plt.close()


1012


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:73: UserWarning: Attempting to set identical bottom == top == 0.0 results in singular transformations; automatically expanding.
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:73: UserWarning: Attempting to set identical bottom == top == 0.0 results in singular transformations; automatically expanding.


1082


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:73: UserWarning: Attempting to set identical bottom == top == 0.0 results in singular transformations; automatically expanding.
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:73: UserWarning: Attempting to set identical bottom == top == 0.0 results in singular transformations; automatically expanding.
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:73: UserWarning: Attempting to set identical bottom == top == 0.0 results in singular transformations; automatically expanding.
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:73: UserWarning: Attempting to set identical bottom == top == 0.0 results in singular transformations; automatically expanding.
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:73: UserWarning: Attempting to set identical bottom == top == 0.0 results in singular transformations; automatically expanding.
/usr/local/lib/python3.6/site-packages/ipykernel_launch

1022


...

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:73: UserWarning: Attempting to set identical bottom == top == 0.0 results in singular transformations; automatically expanding.
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:73: UserWarning: Attempting to set identical bottom == top == 0.0 results in singular transformations; automatically expanding.
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:73: UserWarning: Attempting to set identical bottom == top == 0.0 results in singular transformations; automatically expanding.
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:73: UserWarning: Attempting to set identical bottom == top == 0.0 results in singular transformations; automatically expanding.
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:73: UserWarning: Attempting to set identical bottom == top == 0.0 results in singular transformations; automatically expanding.
/usr/local/lib/python3.6/site-packages/ipykernel_launch

2041


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:73: UserWarning: Attempting to set identical bottom == top == 0.0 results in singular transformations; automatically expanding.
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:73: UserWarning: Attempting to set identical bottom == top == 0.0 results in singular transformations; automatically expanding.
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:73: UserWarning: Attempting to set identical bottom == top == 0.0 results in singular transformations; automatically expanding.
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:73: UserWarning: Attempting to set identical bottom == top == 0.0 results in singular transformations; automatically expanding.
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:73: UserWarning: Attempting to set identical bottom == top == 0.0 results in singular transformations; automatically expanding.
/usr/local/lib/python3.6/site-packages/ipykernel_launch

1021


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:73: UserWarning: Attempting to set identical bottom == top == 0.0 results in singular transformations; automatically expanding.
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:73: UserWarning: Attempting to set identical bottom == top == 0.0 results in singular transformations; automatically expanding.
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:73: UserWarning: Attempting to set identical bottom == top == 0.0 results in singular transformations; automatically expanding.
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:73: UserWarning: Attempting to set identical bottom == top == 0.0 results in singular transformations; automatically expanding.
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:73: UserWarning: Attempting to set identical bottom == top == 0.0 results in singular transformations; automatically expanding.
/usr/local/lib/python3.6/site-packages/ipykernel_launch